In [2]:
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy import stats

In [3]:
def gauss_2D_sample(m, c, n=500):   # 构造双峰高斯分布
    return np.random.multivariate_normal(m, c, size=n)


def plot_sample(m, c, ps=None):   # 高斯分布图像化
    if ps is None:
        ps = gauss_2D_sample(m, c)
#     axis.scatter([p[0] for p in ps], [p[1] for p in ps])
    return ps


def plot_final_result(ps1,ps2,ps, theta):
    four_theta = [0, int(len(theta)/5), int(2*len(theta)/5), int(3*len(theta)/5), int(4*len(theta)/5), len(theta)-1]
    for index in four_theta:
        theta_temp = theta[index]
        theta1 = theta_temp[0][0]
        theta2 = theta_temp[1][0]
        plt.scatter([p[0] for p in ps1], [p[1] for p in ps1])
        plt.scatter([p[0] for p in ps2], [p[1] for p in ps2])
        xx = np.linspace(ps[:, 0].min(), ps[:, 0].max(), 100)
        yy = -(theta2 / theta1) * xx

        plt.plot(xx, yy)
        plt.xlim(-8, 8, 1)
        plt.ylim(-8, 8, 1)
        plt.show()

In [7]:
def neff(weights):
    return 1. / np.sum(np.square(weights))

def sigmoid(theta, x):
    return 1 / (1 + np.exp(-(x @ theta)))

def PF(x, y, beta, R):
    sample_num = 100
    th_conv = np.zeros([2, 2*N])
    
    th_sample = np.random.randn(sample_num,2)
    w = np.ones((sample_num,1))
    for t in range(0, 2*N):
        xn = x[t].reshape((2, 1))
        for i in range(sample_num):
            th_sample[i] = th_sample[i] + beta*np.random.randn(1,2)
            w[i] = stats.norm(loc=sigmoid(th_sample[i], xn.T), scale=R).pdf(y[t])*w[i]
        w += 1.e-300
        w = w/(np.sum(w))
        
        if neff(w) < sample_num:
            w_resample = np.cumsum(w)
            w_resample[-1] = 1.  # avoid round-off error
            indexes = np.searchsorted(w_resample, np.random.rand(sample_num))
            # resample according to indexes
            th_sample = th_sample[indexes]
            w.fill(1.0 / sample_num)
        
        th_conv_t = (th_sample.T @ w)
        th_conv[0,t] = th_conv_t[0]
        th_conv[1,t] = th_conv_t[1]
    
    return th_conv

In [8]:
m1 = [-3, 3]
c1 = [[2, 1], [1, 2]]
p1 = 0.5

m2 = [3, -3]
c2 = [[2, 1], [1, 2]]
p2 = 0.5

xlim, ylim = [-8, 8], [-8, 8]

In [9]:
N = 1000

data_cluster1 = np.array(plot_sample(m1, c1))
d_1 = data_cluster1
data_cluster1 = np.c_[data_cluster1, np.zeros(len(data_cluster1))]


data_cluster2 = np.array(plot_sample(m2, c2))
d_2 = data_cluster2
data_cluster2 = np.c_[data_cluster2, np.ones(len(data_cluster2))]

data_clusters = np.concatenate((data_cluster1, data_cluster2), axis=0)

theta_cov = PF(data_clusters[:, :2], data_clusters[:, -1], 0.01, 1)
print(theta_cov)

IndexError: index 1000 is out of bounds for axis 0 with size 1000